In [1]:
import os
import sys

sys.path.append("../")
adaptive_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if adaptive_root not in sys.path:
    sys.path.append(adaptive_root)

In [2]:
import logging
from copy import deepcopy

import numpy as np
from cmdstanpy import CmdStanModel
from IPython.display import display
from simulation import estimators, fitters, results, simulators, utils

from adaptive.simulation import aggregators

logger = logging.getLogger("cmdstanpy")
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

ModuleNotFoundError: No module named 'adaptive'

In [ ]:
exposed_only = estimators.ExposedOnlyEstimator()
difference = estimators.DifferenceEstimator()

model = CmdStanModel(stan_file="../basic_mlm.stan")
posterior_mean_fitter = fitters.StanFitterBasic(model)

posterior_mean = estimators.PosteriorMeanEstimator(posterior_mean_fitter)

In [ ]:
simulator1 = deepcopy(utils.CHICK_SIMULATOR)
simulator1.identifier = {"p": "p = constant 0.5"}
simulator1.mu_b = 0

simulator2 = deepcopy(utils.CHICK_SIMULATOR)
simulator2.identifier = {"p": "p = 0.5 to 0.95"}
simulator2.p = np.linspace(0.5, 0.95, utils.CHICK_J)
simulator2.mu_b = 0

In [ ]:
sims = []
for simulator in [simulator1, simulator2]:
    for sigma_b in utils.CHICK_SIGMA_B_GRID:
        new_sim = deepcopy(simulator)
        new_sim.sigma_b = sigma_b
        new_sim.identifier = new_sim.identifier | {"sigma_b": sigma_b}
        sims.append(new_sim)

In [ ]:
aggregator = aggregators.SimulationAggregator(sims)
result = aggregator.simulate(
    estimators=[exposed_only, difference, posterior_mean],
    num_reps=100,
    alpha=0.05,
    show_progress=False,
)

In [ ]:
result_means = result.means()
display(result_means)

In [ ]:
import plotly.express as px

# define dash styles for the two p‐settings
dash_map = {"p = constant 0.5": "solid", "p = 0.5 to 0.95": "dash"}

fig_rmse = px.line(
    result_means,
    x="sigma_b",
    y="sample_rmse",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="RMSE vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "pop_rmse": "RMSE"},
)

# keep legend showing only estimator
fig_rmse.update_layout(legend_title_text="Estimator")
fig_rmse.show()

In [ ]:
fig_type_s = px.line(
    result_means,
    x="sigma_b",
    y="type_s_rate",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="Type S error rate vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "type_s_error": "Type S Error Rate"},
)

# keep legend showing only estimator
fig_type_s.update_layout(legend_title_text="Estimator")
fig_type_s.show()

In [ ]:
fig_corr = px.line(
    result_means,
    x="sigma_b",
    y="rank_corr",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="Rank Correlation vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "rank_corr": "Rank Correlation"},
)

# keep legend showing only estimator
fig_corr.update_layout(legend_title_text="Estimator")
fig_corr.show()